In [1]:
from PIL import Image, ImageChops
import argparse
import glob
import numpy as np
import os

In [2]:
# Dataset operations

In [3]:
RAW_WIDTH = 640
RAW_HEIGHT = 480

In [4]:
def read_label_file(path):
    with open(path) as f:
        xys = []
        has_nan = False
        for l in f:
            x, y = map(float, l.split())
            # XXX some bounding boxes has nan coordinates
            if np.isnan(x) or np.isnan(y):
                has_nan = True
            xys.append((x, y))
            if len(xys) % 4 == 0 and len(xys) / 4 >= 1:
                if not has_nan:
                    yield xys[-4], xys[-3], xys[-2], xys[-1]
                has_nan = False

In [5]:
def convert_pcd(path):
    with open(path) as f:
        # move pass the header
        # http://pointclouds.org/documentation/tutorials/pcd_file_format.php
        for _ in range(11):
            f.readline()
            pass
        im = np.zeros((RAW_HEIGHT, RAW_WIDTH), dtype='f4')
        for l in f:
            d, i = l.split()[-2:]
            d = float(d)
            i = int(i)
            x = i % RAW_WIDTH
            y = i % RAW_HEIGHT
            im[y, x] = max(0., d)
        return im

In [6]:
def crop_image(img, box, crop_size):
    cx, cy = np.mean(box, axis=0)
    (x1, y1), (x2, y2) = box[:2]
    # center the image to the bounding box
    o = ImageChops.offset(img, int(RAW_WIDTH/2-cx), int(RAW_HEIGHT/2-cy))
    # rotate the gripper axis to the x-axis
    r = o.rotate(np.rad2deg(np.arctan2(y2 - y1, x2 - x1)))
    # crop the image to a fixed size around the bounding box
    return r.crop((RAW_WIDTH/2-crop_size/2, RAW_HEIGHT/2-crop_size/2,
    RAW_WIDTH/2+crop_size/2, RAW_HEIGHT/2+crop_size/2))

In [7]:
# Preprocessing

In [8]:
CROP_SIZE = 128
# strange scale in the depth data
DEPTH_SCALE_FACTOR = 1e40

In [9]:
DATA_DIR = os.getcwd().rsplit('/', 1)[0] + '/data'

In [10]:
DATA_DIR

'/home/jaichitra.balakrishnan/dl_project/DSCI6005-FinalProject/data'

In [11]:
parser = argparse.ArgumentParser()
parser.add_argument('dataset_path')
parser.add_argument('processed_dataset_path')

#args = parser.parse_args()
args = parser.parse_args([DATA_DIR, DATA_DIR+"/processedData"])

In [12]:
# set up folders
try:
    os.mkdir(args.processed_dataset_path)
except:
    pass
try:
    os.mkdir('%s/pos1' % args.processed_dataset_path)
    os.mkdir('%s/neg1' % args.processed_dataset_path)
except:
    pass

In [13]:
obj_cat = {}
with open('%s/processedData/z.txt' % args.dataset_path) as description_f:
    for line in description_f:
        sid, obj_id, category = line.split()[:3]
        obj_cat[sid] = (obj_id, category)

In [14]:
# file format string
# <pos|neg>/<original image id>-<bounding box id>.<png|tiff>
filename_format = '%s-%03i'

In [15]:
dimg = Image.new('F', (RAW_WIDTH, RAW_HEIGHT))
n_img = 0
n_pos = 0
n_neg = 0
objs = set()
cats = set()

In [16]:
for path in glob.glob('%s/*/pcd*[0-9].txt' % args.dataset_path):    
    n_img += 1
    sample_id = path[-len('1234.txt'):-len('.txt')]    
    objs.add(obj_cat[sample_id][0])
    cats.add(obj_cat[sample_id][1])    
    dim = convert_pcd(path) 
    dimg.putdata(np.nan_to_num(dim.flatten() * DEPTH_SCALE_FACTOR))
    with Image.open(path[:-len('.txt')]+'r.png') as cimg:
        # positive grasps
        for i, box in enumerate(read_label_file(path[:-len('.txt')]+'cpos.txt')):
            n_pos += 1
            filename = filename_format % (sample_id, i)
            crop_image(cimg, box, CROP_SIZE).save('%s/pos/%s.png' % (args.processed_dataset_path, filename))
            np.save('%s/pos1/%s.npy' % (args.processed_dataset_path, filename), np.reshape(crop_image(dimg, box, CROP_SIZE).getdata(), (CROP_SIZE, CROP_SIZE)))
            np.savetxt('%s/pos1/%s.txt'% (args.processed_dataset_path, filename), ['%s,%s' %(obj_cat[sample_id][0], obj_cat[sample_id][1])], fmt='%s')
            
        # negative grasps
        for i, box in enumerate(read_label_file(path[:-len('.txt')]+'cneg.txt')):
            n_neg += 1
            filename = filename_format % (sample_id, i)
            crop_image(cimg, box, CROP_SIZE).save('%s/neg/%s.png' % (args.processed_dataset_path, filename))
            np.save('%s/neg1/%s.npy' % (args.processed_dataset_path, filename), np.reshape(crop_image(dimg, box, CROP_SIZE).getdata(), (CROP_SIZE, CROP_SIZE)))
            np.savetxt('%s/neg1/%s.txt'% (args.processed_dataset_path, filename), ['%s,%s' %(obj_cat[sample_id][0], obj_cat[sample_id][1])], fmt='%s')


n_grasp = n_pos + n_neg
print ('Dataset statistics:')
print ('# of objects:', len(objs))
print ('# of object categories:', len(cats))
print ('# of images:', n_img)
print ('# of labeled grasps: %i positive: %i (%.2f) negative: %i (%.2f)' % (n_grasp, n_pos, n_pos * 1./n_grasp, n_neg, n_neg * 1./n_grasp))

Dataset statistics:
# of objects: 244
# of object categories: 93
# of images: 885
# of labeled grasps: 8019 positive: 5110 (0.64) negative: 2909 (0.36)
